In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.16.0 to work with agogemls


In [3]:
model = ws.models['TitanicAutoML']
print(model.name, 'version', model.version)

TitanicAutoML version 4


In [4]:
import os

folder_name = 'titanic_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_titanic.py")

titanic_service folder created.


In [5]:
%%writefile $script_file
import json
import joblib
import pandas
import time
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('TitanicAutoML')
    model = joblib.load(model_path)
    
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))

# Called when a request is received
def run(raw_data):
    try:
        # Get the input data as a pandas df
        data = json.loads(raw_data)['data']
        data = pandas.read_json(data, orient='records')

        # Get a prediction from the model
        predictions = model.predict(data)
        # Get the corresponding classname for each prediction (0 or 1)
        classnames = ['not-survived', 'survived']
        predicted_classes = []
        for prediction in predictions:
            predicted_classes.append(classnames[prediction])
        
        # Log the input and output data to appinsights:
        info = {
            "input": raw_data,
            "output": predictions.tolist()
            }
        # Return the predictions as JSON
        return json.dumps(predicted_classes)
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

Overwriting ./titanic_service/score_titanic.py


In [5]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

Name AzureML-AutoML
Name AzureML-PyTorch-1.0-GPU
Name AzureML-Scikit-learn-0.20.3
Name AzureML-TensorFlow-1.12-CPU
Name AzureML-PyTorch-1.2-GPU
Name AzureML-TensorFlow-2.0-GPU
Name AzureML-TensorFlow-2.0-CPU
Name AzureML-Chainer-5.1.0-GPU
Name AzureML-TensorFlow-1.13-CPU
Name AzureML-Minimal
Name AzureML-Chainer-5.1.0-CPU
Name AzureML-PyTorch-1.4-GPU
Name AzureML-PySpark-MmlSpark-0.15
Name AzureML-PyTorch-1.3-CPU
Name AzureML-PyTorch-1.1-GPU
Name AzureML-TensorFlow-1.10-GPU
Name AzureML-PyTorch-1.2-CPU
Name AzureML-TensorFlow-1.13-GPU
Name AzureML-Hyperdrive-ForecastDNN
Name AzureML-TensorFlow-1.10-CPU
Name AzureML-PyTorch-1.3-GPU
Name AzureML-PyTorch-1.4-CPU
Name AzureML-Tutorial
Name AzureML-PyTorch-1.0-CPU
Name AzureML-PyTorch-1.1-CPU
Name AzureML-TensorFlow-1.12-GPU
Name AzureML-VowpalWabbit-8.8.0
Name AzureML-Designer-VowpalWabbit
Name AzureML-TensorFlow-2.2-GPU
Name AzureML-TensorFlow-2.2-CPU
Name AzureML-PyTorch-1.6-CPU
Name AzureML-PyTorch-1.6-GPU
Name AzureML-Triton
Name Azure

In [6]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.get(ws, "AzureML-AutoML")

env_file = os.path.join(experiment_folder,"titanic_env.yml")
with open(env_file,"w") as f:
    f.write(env.python.conda_dependencies.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())


Saved dependency info in ./titanic_service/titanic_env.yml
channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.16.0
  - azureml-pipeline-core==1.16.0
  - azureml-telemetry==1.16.0
  - azureml-defaults==1.16.0
  - azureml-interpret==1.16.0
  - azureml-automl-core==1.16.0
  - azureml-automl-runtime==1.16.0
  - azureml-train-automl-client==1.16.0
  - azureml-train-automl-runtime==1.16.0
  - azureml-dataset-runtime==1.16.0
  - inference-schema
  - py-cpuinfo==5.0.0
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_912ebce86aa851a4f789bc2c01e320a3



In [13]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Model

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "titanic"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [14]:
for webservice_name in ws.webservices:
    print(webservice_name)

titanic


In [ ]:
service.update(enable_app_insights=True)

In [15]:
import json
import pandas

test_df = pandas.read_csv('test.csv')
test =test_df.drop(['PassengerId'], axis=1)

# Convert the array to a serializable list in a JSON document
test_json = test[0:1].to_json()
input_json = json.dumps({"data": test_json})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

not-survived


In [16]:
print(service.get_logs())

2020-10-25T11:14:38,906121137+00:00 - iot-server/run 
2020-10-25T11:14:38,907671263+00:00 - gunicorn/run 
2020-10-25T11:14:38,918898557+00:00 - rsyslog/run 
2020-10-25T11:14:38,932286187+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_912ebce86aa851a4f789bc2c01e320a3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_912ebce86aa851a4f789bc2c01e320a3/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_912ebce86aa851a4f789bc2c01e320a3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_912ebce86aa851a4f789bc2c01e320a3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_912ebce86aa851a4f789bc2c01e320a3/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [17]:
endpoint = service.scoring_uri
print(endpoint)

http://8fa1f181-3eac-4392-aca9-3bd35c3209ea.westus.azurecontainer.io/score


In [24]:
import requests

test_json = test[0:5].to_json()
input_json = json.dumps({"data": test_json})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

predicted_classes

['not-survived', 'not-survived', 'not-survived', 'not-survived', 'survived']

In [25]:
service.delete()
print ('Service deleted.')

Service deleted.
